In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), # تحويل الصور إلى Tensor
    transforms.Normalize((0.1307,),(0.3081,))   # تطبيع البيانات
    ])

In [5]:
train_dataset = datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_dataset = datasets.MNIST(root='./data',train=False,download=True,transform=transform)

In [6]:
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=1000,shuffle=False)

In [7]:
class SimpleNN(nn.Module):
  def __init__(self):
    super(SimpleNN,self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(28*28,128) # طبقة مخفية مع 128 وحدة
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(128,10) # طبقة الإخراج (10 أرقام)
  def forward(self,x):
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

model = SimpleNN()

In [8]:
# 3. تحديد دالة الخسارة و المحسن (optimizer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [9]:
def train(model,device,train_loader,optimizer,criterion,epoch):
  model.train()
  for batch_idx,(data,target) in enumerate(train_loader):
    data,target = data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0 :
      print(f"Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]  Loss: {loss.item():.4f}")

In [10]:
def test(model,device,test_loader,criterion):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data,target in test_loader:
      data,target = data.to(device),target.to(device)
      output = model(data)
      test_loss += criterion(output,target).item() * data.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(target).sum().item()
  test_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)
  print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
# 7. تنفيذ التدريب والتقييم
for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

Epoch 1 [0/60000]  Loss: 2.3291
Epoch 1 [6400/60000]  Loss: 0.3328
Epoch 1 [12800/60000]  Loss: 0.2671
Epoch 1 [19200/60000]  Loss: 0.1889
Epoch 1 [25600/60000]  Loss: 0.3004
Epoch 1 [32000/60000]  Loss: 0.1624
Epoch 1 [38400/60000]  Loss: 0.1384
Epoch 1 [44800/60000]  Loss: 0.1365
Epoch 1 [51200/60000]  Loss: 0.1384
Epoch 1 [57600/60000]  Loss: 0.1079
Test set: Average loss: 0.1375, Accuracy: 9592/10000 (95.92%)
Epoch 2 [0/60000]  Loss: 0.0990
Epoch 2 [6400/60000]  Loss: 0.1251
Epoch 2 [12800/60000]  Loss: 0.0424
Epoch 2 [19200/60000]  Loss: 0.1576
Epoch 2 [25600/60000]  Loss: 0.1558
Epoch 2 [32000/60000]  Loss: 0.1086
Epoch 2 [38400/60000]  Loss: 0.0342
Epoch 2 [44800/60000]  Loss: 0.1755
Epoch 2 [51200/60000]  Loss: 0.0451
Epoch 2 [57600/60000]  Loss: 0.0778
Test set: Average loss: 0.0987, Accuracy: 9716/10000 (97.16%)
Epoch 3 [0/60000]  Loss: 0.0505
Epoch 3 [6400/60000]  Loss: 0.0864
Epoch 3 [12800/60000]  Loss: 0.0998
Epoch 3 [19200/60000]  Loss: 0.0704
Epoch 3 [25600/60000]  Loss